# 1D CNN USING GLOVE

In [ ]:

import time
import pandas as pd
import urllib.request
from tika import parser # pip install tika
import requests
from bs4 import BeautifulSoup
import re

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stopwords_eng = set(stopwords.words("english"))
snowball = SnowballStemmer(language="english")
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping

from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.preprocessing.text import Tokenizer

import tensorflow
from tensorflow import keras
#import torchtext
import matplotlib.pyplot as plt
%matplotlib inline
import gc

In [ ]:
#ff: fast forward to this point
merged_pdf_df = pd.read_csv (r'merged_pdf_df_clean.csv')

In [ ]:
merged_pdf_df

In [ ]:
#the above three are from source
# OR run this one
rs = 420
maxlen = 500 #state how long the sequences should be - anything longer that this gets cut
word_num = 500
#embedding_dim = 50
embedding_dim = 128
x_y_xplit = 0.25


train_dataset = merged_pdf_df.sample(frac=0.8,random_state=50) #random state is a seed value
test_dataset  = merged_pdf_df.drop(train_dataset.index)

X_test_text = list(test_dataset['clean_text_tok'])
Y_test = list(test_dataset['pnn_class'])
X_train_text = list(train_dataset['clean_text_tok'])
Y_train = list(train_dataset['pnn_class'])

unique_classes = list(set(Y_train))
target_classes = ["UP", "DOWN", "SAME"]

from sklearn.preprocessing import LabelEncoder
labenc = LabelEncoder()
Y_train = labenc.fit_transform(Y_train)
Y_test = labenc.fit_transform(Y_test)

## Subtracted 1 from labels to bring range from 1-4 to 0-3
#Y_train, Y_test = np.array(Y_train), np.array(Y_test)

In [ ]:
print(len(train_dataset))
print(len(test_dataset))

In [ ]:
tokenizer = Tokenizer(num_words=word_num)
tokenizer.fit_on_texts(X_train_text)

X_train = tokenizer.texts_to_sequences(X_train_text)
X_test = tokenizer.texts_to_sequences(X_test_text)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(X_train[2])

In [ ]:


X_train = pad_sequences(X_train, padding='post', maxlen=maxlen) #post padding so 0s get added to the end, not start
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50
#this must equal 50 because glove file is 50 dim

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath,encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            print(word)
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
glove_file = 'glove.6B.50d.txt'
embedding_matrix = create_embedding_matrix(glove_file,tokenizer.word_index, embedding_dim )


In [ ]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size
#51.9% of words in content are covered by model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense
from keras import backend as K 
from sklearn.metrics import accuracy_score, classification_report

from sklearn.metrics import confusion_matrix
import scikitplot as skplt
import matplotlib.pyplot as plt
from lime import lime_text


#### setting all seeds for reproducible results
seed_value= 420
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.random.set_seed(seed_value)

In [ ]:
#function to plot results
import matplotlib.pyplot as plt

def plot_loss_and_acc(history):
    train_loss = history.history["loss"]
    train_acc = history.history["accuracy"]
    val_loss = history.history["val_loss"]
    val_acc = history.history["val_accuracy"]

    fig = plt.figure(figsize=(12,5))

    ax = fig.add_subplot(121)
    ax.plot(range(len(train_loss)), train_loss, label="Train Loss");
    ax.plot(range(len(val_loss)), val_loss, label="Validation Loss");
    plt.xlabel("Epochs"); plt.ylabel("Loss");
    plt.title("Train Loss vs Validation Loss");
    plt.legend(loc="best");

    ax = fig.add_subplot(122)
    ax.plot(range(len(train_acc)), train_acc, label="Train Accuracy");
    ax.plot(range(len(val_acc)), val_acc, label="Validation Accuracy");
    plt.xlabel("Epochs"); plt.ylabel("Accuracy");
    plt.title("Train Accuracy vs Validation Accuracy");
    plt.legend(loc="best");  

In [ ]:

from sklearn.metrics import confusion_matrix
import scikitplot as skplt
import matplotlib.pyplot as plt

skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=True,
                                    title="Confusion Matrix",
                                    cmap="Greens",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);



### THIS USES CATEGORICAL CROSSENTROPY 
### IN ORDER TO DO SO, WE NEED TO CONVERT THE HOTENCODED LABELS
### HENCE WHY THIS IS IN A DIFFERENT SPOT

In [ ]:
#### setting all seeds for reproducible results
seed_value= 420
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.random.set_seed(seed_value)

################################################

def conv1d_func(embed_len,filt,kern,pad,act,epoch,btch_siz,esmon,esmod,espat):

    Y_train_cat = to_categorical(Y_train, 3)
    Y_test_cat = to_categorical(Y_test, 3)

    embed_len = embed_len

    inputs = Input(shape=(word_num, ))
    embeddings_layer = Embedding(input_dim=vocab_size
                                 ,weights=[embedding_matrix]
                                 ,output_dim=embed_len
                                 ,input_length=maxlen)
    conv = Conv1D(filt, kern, padding=pad) ## Channels last
    dense = Dense(len(target_classes), activation=act)

 
    
    x = embeddings_layer(inputs)
    x = conv(x)
    x = tensorflow.reduce_max(x, axis=1)
    output = dense(x)

    model = Model(inputs=inputs, outputs=output)

    model.summary()

    model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

    es = EarlyStopping(monitor=esmon, mode=esmod,patience=espat, verbose=1)

    history = model.fit(X_train, Y_train_cat,
                        epochs=epoch,
                        verbose=False,
                        validation_data=(X_test, Y_test_cat),
                        batch_size=btch_siz, 
                        callbacks=[es])
    
    loss, accuracy = model.evaluate(X_train, Y_train_cat, verbose=True)
    print("Training Accuracy: {:.4f}".format(accuracy))
    loss, accuracy = model.evaluate(X_test, Y_test_cat, verbose=True)
    print("Testing Accuracy:  {:.4f}".format(accuracy))


    #========================
    plot_loss_and_acc(history)    
    #========================


    from sklearn.metrics import accuracy_score, classification_report

    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
    print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
    print("\nClassification Report : ")
    print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes))


    skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                        normalize=False,
                                        title="Confusion Matrix",
                                        cmap="Greens",
                                        hide_zeros=True,
                                        figsize=(8,8)
                                        );
    plt.xticks(rotation=90);
    
    K.clear_session() #this clears the session to forget previous model executions and become overfitted

In [ ]:
conv1d_func(embed_len=50,filt=80,kern=40,pad="valid",act="softmax",epoch=20,btch_siz=500,esmon="val_loss",esmod="auto",espat=2)


### CREATING A SECOND CNN FUNCTION - TWO LAYERS

In [ ]:
#categorical crossentropy requires - surprise surprise - categorical variables
#winner winner
Y_train_cat = to_categorical(Y_train, 3)
Y_test_cat = to_categorical(Y_test, 3)

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size
                                 ,weights=[embedding_matrix]
                                 ,output_dim=50
                                 ,input_length=maxlen
                                 ,trainable=True))

model.add(layers.Conv1D(50, 60, activation='softmax'))
model.add(layers.GlobalMaxPool1D())
model.add(Dense(len(target_classes), activation="sigmoid"))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


es = EarlyStopping(monitor="val_loss", mode='auto',patience=3, verbose=1)

history = model.fit(X_train, Y_train_cat,
                    epochs=32,
                    verbose=False,
                    validation_data=(X_test, Y_test_cat),
                    batch_size=250, 
                    callbacks=[es])
    
loss, accuracy = model.evaluate(X_train, Y_train_cat, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test_cat, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_loss_and_acc(history)  


train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
print("\nClassification Report : ")
print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes))


skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=False,
                                    title="Confusion Matrix",
                                    cmap="Greens",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);

K.clear_session() #this clears the session to forget previous model executions and become overfitted

In [ ]:
from lime import lime_text

explainer = lime_text.LimeTextExplainer(class_names=target_classes, verbose=True)

explainer

def make_predictions(X_batch_text):
    X_batch = pad_sequences(tokenizer.texts_to_sequences(X_batch_text), maxlen=500, padding="post", truncating="post", value=0)
    preds = model.predict(X_batch)
    return preds



In [ ]:
rng = np.random.RandomState(30)
idx = rng.randint(1, len(X_test))

print("Prediction : ", target_classes[model.predict(X_test[idx:idx+1]).argmax(axis=-1)[0]])
print("Actual :     ", target_classes[Y_test[idx]])

explanation = explainer.explain_instance(X_test_text[idx], classifier_fn=make_predictions, labels=Y_test[idx:idx+1], num_features=15)
explanation.show_in_notebook()

In [ ]:
from lime import lime_text

explainer = lime_text.LimeTextExplainer(class_names=target_classes, verbose=True)

def make_predictions(X_batch_text):
    X_batch = pad_sequences(tokenizer.texts_to_sequences(X_batch_text), maxlen=500, padding="post", truncating="post", value=0)
    preds = model.predict(X_batch)
    return preds

def lime_run(i):
    rng = np.random.RandomState(i)
    idx = rng.randint(1, len(X_test))

    print("Prediction : ", target_classes[model.predict(X_test[idx:idx+1]).argmax(axis=-1)[0]])
    print("Actual :     ", target_classes[Y_test[idx]])

    explanation = explainer.explain_instance(X_test_text[idx], classifier_fn=make_predictions, labels=Y_test[idx:idx+1], num_features=15)
    explanation.show_in_notebook()

In [ ]:
for i in range(400,450):
    lime_run(i)

In [ ]:
#categorical crossentropy requires - surprise surprise - categorical variables
#winner winner
Y_train_cat = to_categorical(Y_train, 3)
Y_test_cat = to_categorical(Y_test, 3)

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size
                                 ,weights=[embedding_matrix]
                                 ,output_dim=50
                                 ,input_length=maxlen
                                 ,trainable=True))

model.add(layers.Conv1D(80, 40, activation='softmax'))
model.add(layers.GlobalMaxPool1D())
model.add(Dense(len(target_classes), activation="tanh"))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


es = EarlyStopping(monitor="val_loss", mode='auto',patience=3, verbose=1)

history = model.fit(X_train, Y_train_cat,
                    epochs=32,
                    verbose=False,
                    validation_data=(X_test, Y_test_cat),
                    batch_size=250, 
                    callbacks=[es])
    
loss, accuracy = model.evaluate(X_train, Y_train_cat, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test_cat, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_loss_and_acc(history)  


train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
print("\nClassification Report : ")
print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes))


skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=False,
                                    title="Confusion Matrix",
                                    cmap="Greens",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);

K.clear_session() #this clears the session to forget previous model executions and become overfitted

In [ ]:
#categorical crossentropy requires - surprise surprise - categorical variables
Y_train_cat = to_categorical(Y_train, 3)
Y_test_cat = to_categorical(Y_test, 3)

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size
                                 ,weights=[embedding_matrix]
                                 ,output_dim=50
                                 ,input_length=maxlen
                                 ,trainable=True))

model.add(layers.Conv1D(30, 80, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(Dense(len(target_classes), activation="tanh"))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


es = EarlyStopping(monitor="val_loss", mode='auto',patience=3, verbose=1)

history = model.fit(X_train, Y_train_cat,
                    epochs=32,
                    verbose=False,
                    validation_data=(X_test, Y_test_cat),
                    batch_size=250, 
                    callbacks=[es])
    
loss, accuracy = model.evaluate(X_train, Y_train_cat, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test_cat, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_loss_and_acc(history)  


train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print("Train Accuracy : {}".format(accuracy_score(Y_train, np.argmax(train_preds, axis=1))))
print("Test  Accuracy : {}".format(accuracy_score(Y_test, np.argmax(test_preds, axis=1))))
print("\nClassification Report : ")
print(classification_report(Y_test, np.argmax(test_preds, axis=1), target_names=target_classes))


skplt.metrics.plot_confusion_matrix([target_classes[i] for i in Y_test], [target_classes[i] for i in np.argmax(test_preds, axis=1)],
                                    normalize=False,
                                    title="Confusion Matrix",
                                    cmap="Greens",
                                    hide_zeros=True,
                                    figsize=(8,8)
                                    );
plt.xticks(rotation=90);

K.clear_session() #this clears the session to forget previous model executions and become overfitted